In [2]:
!nvidia-smi

Wed Aug 17 12:34:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%cd /content/
!rm -rf cloth-segmentation
!git clone https://github.com/levindabhi/cloth-segmentation.git
%cd cloth-segmentation
!gdown --id 1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ
!mkdir input_images
!mkdir output_images

/content
Cloning into 'cloth-segmentation'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 62 (delta 3), reused 6 (delta 0), pack-reused 52
Unpacking objects: 100% (62/62), done.
/content/cloth-segmentation
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ
To: /content/cloth-segmentation/cloth_segm_u2net_latest.pth
100% 177M/177M [00:03<00:00, 48.9MB/s]


In [4]:
!cp -r '/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/Chanel/Fall Couture Collection' /content/cloth-segmentation/input_images

# Upload input images in *input_images* folder.



In [5]:
import os
from tqdm import tqdm
# from tqdm.notebook import tqdm
from PIL import Image
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from data.base_dataset import Normalize_image
from utils.saving_utils import load_checkpoint_mgpu

from networks import U2NET
device = 'cuda'

image_dir = 'input_images/Fall Couture Collection'
result_dir = 'output_images'
checkpoint_path = 'cloth_segm_u2net_latest.pth'

def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
net = net.to(device)
net = net.eval()

palette = get_palette(4)

!rm -rf input_images/.ipynb_checkpoints
### edited by jemu
dirs = sorted(os.listdir(image_dir))

sw=True
cnt = 0
for dir in dirs:
  images_list = sorted(os.listdir(os.path.join(image_dir, dir)))
  year = str([i for i in dir.split() if i.isnumeric()][0])
  for image_name in images_list:
    if image_name[-4:] != ".jpg": continue
    img = Image.open(os.path.join(image_dir, dir, image_name)).convert('RGB')
    img_size = img.size
    img = img.resize((768, 768), Image.BICUBIC)
    image_tensor = transform_rgb(img)

    # edited
    masked_img = transforms.Compose([transforms.ToTensor()])(img).cpu().numpy() 

    image_tensor = torch.unsqueeze(image_tensor, 0)
    
    output_tensor = net(image_tensor.to(device))
    output_tensor = F.log_softmax(output_tensor[0], dim=1)
    output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
    output_tensor = torch.squeeze(output_tensor, dim=0)
    output_tensor = torch.squeeze(output_tensor, dim=0)
    # 0: 배경
    # 1: 상의
    # 2: 하의
    # 3: 원피스
    output_arr = output_tensor.cpu().numpy()
    output_arr = np.where(output_arr != 0, masked_img, 0)
    output_arr = np.transpose(output_arr, (1, 2, 0))
    output_img = Image.fromarray((output_arr*255).astype('uint8'), mode='RGB')
    # output_img = output_img.resize(img_size, Image.BICUBIC)
    
    # output_img.putpalette(palette)
    file_name = year+'_'+str(cnt)
    output_img.save(os.path.join(result_dir, file_name + '_generated.png'))
    cnt += 1

----checkpoints loaded from path: cloth_segm_u2net_latest.pth----


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3722: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [6]:
!rm -rf /content/output_chanel_1.zip
!zip -r /content/output_chanel_1.zip /content/cloth-segmentation/output_images

  adding: content/cloth-segmentation/output_images/ (stored 0%)
  adding: content/cloth-segmentation/output_images/2011_682_generated.png (deflated 1%)
  adding: content/cloth-segmentation/output_images/2013_789_generated.png (deflated 1%)
  adding: content/cloth-segmentation/output_images/2018_1135_generated.png (deflated 3%)
  adding: content/cloth-segmentation/output_images/2013_823_generated.png (deflated 2%)
  adding: content/cloth-segmentation/output_images/2011_698_generated.png (deflated 2%)
  adding: content/cloth-segmentation/output_images/2010_646_generated.png (deflated 2%)
  adding: content/cloth-segmentation/output_images/2013_780_generated.png (deflated 1%)
  adding: content/cloth-segmentation/output_images/2010_628_generated.png (deflated 3%)
  adding: content/cloth-segmentation/output_images/2012_736_generated.png (deflated 1%)
  adding: content/cloth-segmentation/output_images/2006_365_generated.png (deflated 1%)
  adding: content/cloth-segmentation/output_images/2005

# Download results from *output_images*

In [ ]:
!rm -rf output_images/*